**Quantitative X-Ray Imaging: Tutorial 05**   
*Julia Herzen, Franziska Hinterdobler, Sara Savatović, Sami Wirtensohn*   

# Exercise 05: Image-based material decomposition

The last two weeks, we concentrated on material decomposition approaches based on projection images. The other approach is to perform the decomposition using the already reconstructed images which contain information about the local attenuation coefficient $\mu$. The advantage is a less complicated decomposition approach based on the tabulated values for $\mu$ without the need for calibration measurements. In this exercise, we will walk through such a decomposition. As reminder: For a sample containing two materials 1 and 2 in volume fractions of $n_1$ and $n_2$ the following expression for the total attenuation coefficient $\mu_{tot}$ is valid

$
\begin{pmatrix} \mu_\text{tot, HE}(E) \\ \mu_\text{tot, LE}(E)\end{pmatrix} = \begin{pmatrix} \mu_\text{1, HE}(E) & \mu_\text{2, HE}(E)\\ \mu_\text{1, LE}(E) & \mu_\text{2, LE}(E)\end{pmatrix} \cdot \begin{pmatrix}n_1\\n_2\end{pmatrix}
$

This is the linear system of equations we are trying to solve to arrive at the volume fractions. 

## Part 1: Thinking about the source spectrum / detected spectrum

### Simulating the emitted spectrum
When using laboratory sources with a polychromatic spectrum, the choice of acceleration voltage and filters is crucial to obtain two spectra that are reasonably separated. Let's load some simulated spectra from a tungsten source

In [1]:
# We can open our figures in interactive mode
# or as images
#%matplotlib inline
from scipy import interpolate
import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt
plt.ion()
import xraylib
# from pathlib import Path # Path('/etc')

In [2]:
def load_spectrum(kVp):
    '''This function can be used to load tungsten X-ray spectra ranging from 35kVp to 150kVp'''
    data = np.loadtxt(base_path + 'W_'  + str(kVp) + 'kVp.txt', delimiter = ',')
    return data
    

base_path = 'spectrum_data/'

energy_80, intensity_80 = load_spectrum(80)
energy_90, intensity_90 = load_spectrum(90)
energy_100, intensity_100 = load_spectrum(100)
energy_120, intensity_120 = load_spectrum(120)
energy_140, intensity_140 = load_spectrum(140)

plt.figure()
plt.title('Tungsten anode X-ray spectra')
plt.plot(energy_140, intensity_140, energy_80, intensity_80, energy_100, intensity_100, energy_120, intensity_120)
plt.xlabel('Energy in keV')
plt.ylabel('Intensity')
plt.show()

One way to generate two spectrally separated measurements is to introduce a filter to the source during the high energy measurement. Compare the spectra without and with filter. In our example, we will use tin (Zinn). Try at least one other filter material, as well and observe the effect. What specifications are important when choosing a filter?

In [3]:
def get_attenuation_coefficient(compound, density, energy):
    energies = energy
    mues = np.empty_like(energies)
    for i, energy in enumerate(energies):
        mues[i] = xraylib.CS_Total_CP(compound, energy) # cm^2/g
    mue_rho = mues * density # 1/cm
    return mue_rho

In [4]:
# choose your acceleration voltages for the spectra
energy_high = energy_140
energy_low = energy_80
intensity_high = intensity_140
intensity_low = intensity_80

# build filter
mu_rho = get_attenuation_coefficient('Sn', 7.265, energy_high) # Tin (Zinn) with a density of 7.265 g/cm^3
filter_thickness = 0.02 # in cm --> adapt thickness to filter material if not tin

plt.figure()
plt.title('Energy dependent attenuation of Tin')
plt.plot(energy_140, mu_rho)
plt.xlabel('Energy in keV')
plt.ylabel('Linear attenuation coefficient [1/cm]')
plt.yscale('log')
plt.show()

# we can increase the heating voltage for more electrons -> higher intensity in X-ray photons
# now we introduce the filter
intensity_HE_highcurrent = intensity_high * 1.5
intensity_HE_highcurrent_filtered = intensity_HE_highcurrent * np.exp(- mu_rho * filter_thickness) # insert Beer–Lambert law

#plot both spectra with and without filter and compare
plt.figure()
plt.title('Comparison of two spectra')
plt.axis('off')

plt.subplot(211)
plt.plot(energy_high, intensity_HE_highcurrent, label = 'HE')
plt.plot(energy_low, intensity_low, label = 'LE')
plt.ylabel('Intensity')
plt.xlabel('Energy in keV')
plt.legend()
plt.subplot(212)
plt.plot(energy_high, intensity_HE_highcurrent_filtered, label = 'HE')
plt.plot(energy_low, intensity_low, label = 'LE')
plt.xlabel('Energy in keV')
plt.ylabel('Intensity')
plt.legend()
plt.show()


# one measure for how separated the spectra are is to calculate the overlap:
# Spectral overlap of low and filtered high spectrum

intensity_integral_LE = np.sum(intensity_low)
intensity_integral_HE_highcurrent_filtered = np.sum(intensity_HE_highcurrent_filtered)

# e.g. for 80 and 140 keV (with an 1.5 intensity factor) the spectra cross at 52.5 keV -> energy_140[51] = energy_80[51]
# you might have to adjust the overlap position depending on your chosen energies, filters and intensities

# function to find the indices where the spectra cross:
idx = np.argwhere(np.diff(np.sign(intensity_HE_highcurrent_filtered - intensity_low))).flatten() # choose the appropriate crossing point
print('The two spectra cross at', idx)
overlap_integral = np.sum(intensity_HE_highcurrent_filtered[:51]) + np.sum(intensity_low[51:])
overlap_percentage = 100 * overlap_integral / (intensity_integral_LE + intensity_integral_HE_highcurrent_filtered - overlap_integral)

print('The two medical spectra have an overlap of {overlap} %.'.format(overlap=overlap_percentage.round(2)))

# another interesting measure is the mean energy of the spectra:
meanE_low = np.sum(intensity_low * energy_low) / np.sum(intensity_low)
meanE_high = np.sum(intensity_HE_highcurrent * energy_high) / np.sum(intensity_HE_highcurrent)
meanE_high_filtered = np.sum(intensity_HE_highcurrent_filtered * energy_high)/np.sum(intensity_HE_highcurrent_filtered)

print('Mean energy \n low spectrum: {low} keV \n high spectrum: {high} keV \n high and filtered spectrum: {high_fil} keV'.format\
      (low = meanE_low.round(2), high = meanE_high.round(2), high_fil = meanE_high_filtered.round(2)))

The two spectra cross at [  9  51 140]
The two medical spectra have an overlap of 23.14 %.
Mean energy 
 low spectrum: 42.8 keV 
 high spectrum: 57.47 keV 
 high and filtered spectrum: 73.8 keV


### Separating the spectrum using a dual-layer detector

We can also get spectrally separated measurements using a dual-layer detector

We have now seen how filters can affect the spectral overlap and mean energy. ![dual_layer.JPG](attachment:dual_layer.JPG)  
Let's take a look at the theoretical detector response function of such a detector. The detector response is calculated via the quantum efficiency and the energy response. The dector response for the front (f) and rear (r) layers can be described as

$
DR_f(E) = \underbrace{(1-e^{(- \mu_{sc}(E) \cdot d_f)})}_{\text{quantum efficiency}} \\
DR_r(E) = \underbrace{e^{(- \mu_{sc}(E) \cdot d_f)}}_{\text{attenuation front layer}} \cdot (1 - e^{(- \mu_{sc}(E) \cdot d_r)})
$

with $\mu_{sc}$ being the attenuation coefficient of the scintillator material and $d_i$ the respective layer thickness. You can tune the separation of the spectra by adjusting the layer thickness. We will look at different responses around the layer thickness of 0.7mm.

As a simplification, we will assume the quantum efficiency of the second detector layer is 1 and its thickness is enough so all remaining photons are stopped. We will also neglect the linear scaling with the energy for both layers.

In [5]:
mu_znse = get_attenuation_coefficient('ZnSe', 5.27, energy_120) # ZnSe Scintillator layer attenuation coefficient
layer_thickness_f = 0.07 # in cm
layer_thickness_r = 0.07 # in cm

det_response_f = (1-np.exp(-layer_thickness_f*mu_znse)) #* energy_120 # scintillator attenuation in front layer -> detector response
det_response_r = (np.exp(-layer_thickness_f*mu_znse)) #* energy_120 # rear layer -> all remaining photons get absorbed

energy = energy_120 # we will focus on a typical medical spectrum with 120 kVp
intensity = intensity_120
plt.figure('Detector energy response function for DLCT')
plt.plot(energy, det_response_f, 'r-', label='Front Layer')
plt.plot(energy, det_response_r, 'b-', label='Rear Layer')
# we will look at different front layer thickness designs
layer_thickness_var = np.arange(0.07-0.03,0.07+0.03+0.01,0.02)
for thick in layer_thickness_var:
    det_res_f = (1-np.exp(-thick*mu_znse))
    det_res_r = (np.exp(-thick*mu_znse))
    plt.plot(energy, det_res_f, 'r--')
    plt.plot(energy, det_res_r, 'b--')
plt.xlabel('Energy [keV]')
plt.ylabel('Detector energy response')# (0.4, 0.6, 0.7, 0.8, 1.0) mm ZnSe')
plt.legend()
plt.show(block=False)

Next, we see how big the spectral overlap is for a design using a 0.7mm thick front layer. As seen before, check where the spectra cross and calculate the overlap from there. Try different layer thicknesses and observe, how the overlap changes.
Next, we see how big the spectral overlap is for a design using a 0.7mm thick front layer. As seen before, check where the spectra cross and calculate the overlap from there. Try out different layer thicknesses to see if you can improve the overlap.

In [ ]:
plt.figure('Effective spectra in energy discriminating detector')
plt.plot(energy,intensity*det_response_f,'r-',label='Front Layer')
plt.plot(energy,intensity*det_response_r,'b-',label='Rear Layer')
plt.xlabel('Energy [keV]')
plt.ylabel('Detector energy response x Intensity')
plt.legend()
plt.show(block=False)

# Spectral overlap
intensity_integral_f = np.sum(intensity*det_response_f)
intensity_integral_r = np.sum(intensity*det_response_r)

idx = np.argwhere(np.diff(np.sign(intensity*(det_response_f - det_response_r)))).flatten() # choose the appropriate crossing point
print('The two spectra cross at', idx)
# the spectra cross at X -> energy[X]

overlap_integral = np.sum((intensity*det_response_r)[:61]) + np.sum((intensity*det_response_f)[61:])
overlap_percentage = 100 * overlap_integral / (intensity_integral_f + intensity_integral_r - overlap_integral)

print('The spectral overlap in this DL design is {overlap} %.'.format(overlap=overlap_percentage.round(2)))

The two spectra cross at [  9  61 120]
The spectral overlap in this DL design is 30.58 %.


## Part 2: Simulating a phantom and performing a decomposition

### Step 1: Build a phantom 
Build a phantom filled with different attenuation coefficients, e.g. for soft tissue, bone and iodine. You may, for example, reuse the phantom you built in exercise 03.

In [8]:
# use xraylib to get the material data for the phantom
energies = np.arange(20., 140., 10)
soft = get_attenuation_coefficient('Tissue, Soft (ICRP)', 0.92, energy = energies)
water = get_attenuation_coefficient('Water, Liquid', 1.0, energy = energies )
iodine = get_attenuation_coefficient('I', 4.93, energy = energies)
contrast = 0.98 * water + 0.02*iodine
bone = get_attenuation_coefficient('Bone, Compact (ICRU)', 1.92, energy=energies)

# create phantom with bone, tissue and air components
phantom_LE = np.load('shepp_logan.npy')
phantom_HE = np.load('shepp_logan.npy')

# use the known attenuation values to replace the original values in the modified Shepp-Logan phantom
phantom_LE[phantom_LE == 1.0] = bone[np.where(energies == 60.)]
phantom_LE[phantom_LE == 0.19999999] = soft[np.where(energies == 60.)]
phantom_LE[phantom_LE == -1.4901161e-08] = bone[np.where(energies == 60.)]
phantom_LE[phantom_LE == 0.29999998] = bone[np.where(energies == 60.)]
phantom_LE[phantom_LE == 0.0] = 0.0001

phantom_HE[phantom_HE == 1.0] = bone[np.where(energies == 120.)]
phantom_HE[phantom_HE == 0.19999999] = soft[np.where(energies == 120.)]
phantom_HE[phantom_HE == -1.4901161e-08] = bone[np.where(energies == 120.)]
phantom_HE[phantom_HE == 0.29999998] = bone[np.where(energies == 120.)]
phantom_HE[phantom_HE == 0.0] = 0.0001

# choose a slice to decompose, e.g. the middle slice
slice_LE = phantom_LE[:,:,100].copy()
slice_HE = phantom_HE[:,:,100].copy()

In [9]:
# Show the two slices
fig, axs = plt.subplots(1, 2)
pcm1 = axs[0].imshow(slice_LE, cmap='gray', vmin=0.2)
fig.colorbar(pcm1, ax=axs[0], fraction=0.046, pad=0.04)
axs[0].axis('off')
axs[0].title.set_text('Low Energy slice phantom')
pcm2 = axs[1].imshow(slice_HE, cmap='gray', vmin=0.2)
fig.colorbar(pcm2, ax=axs[1], fraction=0.046, pad=0.04)
axs[1].axis('off')
axs[1].title.set_text('High Energy slice phantom')
fig.tight_layout()
plt.show()


### Step 2: Build coefficient matrix
In the beginning of this exercise, we showed a system of equations used for image-based decomposition. We will build the decomposition matrix in the next step.

In [11]:
# build the coefficient matrix for decomposition
# e.g we will decompose into bone and soft tissue
mu_1_LE = bone[np.where(energies == 60.)][0] # bone
mu_1_HE = bone[np.where(energies == 120.)][0]


mu_2_LE = soft[np.where(energies == 60.)][0] # Soft tissue
mu_2_HE = soft[np.where(energies == 120.)][0]

# As we are performing a two material decomposition, the coeff matrix is 2x2
coeff = np.array([[mu_1_LE, mu_2_LE],[mu_1_HE, mu_2_HE]])

### Step 3: Perform decomposition
We've set up a linear system of equations (LSE) in the introduction. You need to solve this LSE for each pixel using the measurements we took and the coefficient matrix we built. You can do so by iterating over every pixel, or make use of numpy. Iterating is a lot slower, so we will use the numpy way.
For this, we need to bring our data in the right shape: one array, where the last axis is the low or high energy pixel value.

In [12]:
measurements = np.zeros((slice_LE.shape[0], slice_LE.shape[1],2)) # build an empty array in the right shape
measurements[:,:, 0] = slice_LE # assign the images to the last axis
measurements[:,:, 1] = slice_HE
print(phantom_LE.shape, measurements.shape)


# This is the step where all the numpy magic happens. 
# If you want to understand the notation, please look at the numpy documentation and make yourself familiar with its array broadcasting rules.
decomp=np.linalg.solve(coeff[None, None], measurements)

# its easier to look at the images individually.
decomp_mat_1 = decomp[:,:,0]
decomp_mat_2 = decomp[:,:,1]

# checking out the decomposition images
#%matplotlib notebook 
fig, axs = plt.subplots(1, 2)
pcm1 = axs[0].imshow(decomp_mat_1, cmap='gray', vmin=0, vmax=1.2)
fig.colorbar(pcm1, ax=axs[0], fraction=0.046, pad=0.04)
axs[0].axis('off')
axs[0].title.set_text('Material Image 1')
pcm2 = axs[1].imshow(decomp_mat_2, cmap='gray', vmin=0, vmax=1.5)
fig.colorbar(pcm2, ax=axs[1], fraction=0.046, pad=0.04)
axs[1].axis('off')
axs[1].title.set_text('Material Image 2')
fig.tight_layout()
plt.show()

(200, 200, 200) (200, 200, 2)


## Part 3: Dual-energy decomposition on microCT data

In this part, we provide you with some exemplary data from a commercial microCT, to perform material decomposition with a conventional flatpanel detector and two measurements at higher and lower energies.
The sample is a microCT hydroxyapatite (Ca5P3O13H) phantom used to calibrate small animal microCTs. There are fixed HA concentrations in the form of rods: 1200, 800, 200, 50, 0 [mg HA/cm³]. The surrounding material is resin, which we will approximate as water.
First, let's get a feeling for the absorption coefficients of these basis materials. We also plot the attenuation of iodine, since it is such a common contrast agent.

In [13]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import xraylib
#import roipoly
#%matplotlib notebook 
#plt.ion()

In [14]:
# choose one of the spectra, e.g. 140 keV
energy = energy_140
mu_rho_water = get_attenuation_coefficient('H2O', 1.0, energy) # linear attenuation coefficient
mu_water = mu_rho_water/1.0 # mass attenuation coefficient (mu_rho/rho)
mu_rho_HA = get_attenuation_coefficient('Ca5P3O13H', 3.14, energy) # Hydroxyapatite with a bone mineral density of 3.14 g/cm^3
mu_HA = mu_rho_HA/3.14
mu_rho_I = get_attenuation_coefficient('I', 4.94, energy) # Iodine contrast
mu_I = mu_rho_I/4.94

plt.figure()
plt.title('Attenuation of water, bone (HA) and iodine')
plt.plot(energy, mu_rho_water, label = 'Water')
plt.plot(energy, mu_rho_HA, label = 'Bone')
plt.plot(energy, mu_rho_I, label = 'Iodine')
plt.xlabel('Energy in keV')
plt.ylabel('Linear attenuation coefficient [1/cm]')
plt.yscale('log')
plt.legend()
plt.show()

### Step 1: Loading the images and correcting them.

The reconstructed images are saved as 16 bit tif files. This means they contain integer values between 0 and 65535. 
During reconstruction the attenuation coefficient mu is reconstructed and then mapped to the tif values. 
Usually, one can find the minimum and maximum value in the accompanying log files as 'min = ' and 'max = '. Since you haven't performed the measurements yourselves, we provide this conversion.

In order to load the tif file we use pillow (or PIL Python Image Library). Directly convert them to a numpy array.
LE is labeling the low energetic spectrum image, HE the high energetic spectrum image.


In [15]:
LE=np.array(Image.open('./MicroCT_data/slice_0420low.tif'))
HE=np.array(Image.open('./MicroCT_data/slice_0420high.tif'))

# Enter the values from the _reco.log files here
LE_min = -0.8815
LE_max = 4.219888

HE_min = -0.3876
HE_max = 0.63236


def tif_to_mu(img, w_min, w_max):
    '''
    Just a small function to recover the mu values in 1/cm.
    The value 2 pi is a necessary, as the reconstruction program omitts this in its filter functions.
    They claim to do this for compatibility reasons
    '''
    a=(w_max-w_min)/65535.
    return (img*a+w_min)

# convert the tif scaled images into mu scaled images
LE_mu=tif_to_mu(LE, LE_min, LE_max)
HE_mu=tif_to_mu(HE, HE_min, HE_max)

In [16]:
#%matplotlib inline
fig, axs = plt.subplots(1, 2)
pcm1 = axs[0].imshow(LE_mu, cmap='gray', vmin=0.2)
fig.colorbar(pcm1, ax=axs[0], fraction=0.046, pad=0.04)
axs[0].axis('off')
axs[0].title.set_text('Low Energy')
pcm2 = axs[1].imshow(HE_mu, cmap='gray', vmin=0)
fig.colorbar(pcm2, ax=axs[1], fraction=0.046, pad=0.04)
axs[1].axis('off')
axs[1].title.set_text('High Energy')
fig.tight_layout()
plt.show()

### Step 2: Build the coefficient matrix for the material decomposition

When we built the coefficient matrix for our phantom, we assumed monochromatic radiation and used the exact theoretical attenuation coefficients. Now, the situation is different: Our spectrum is polychromatic.

You may have noticed that we didn't provide an acceleration voltage or energy range. However, to build our coefficient matrix (see equation at the beginning of the exercise), we need to know the theoretical attenuation coefficients of our basis materials. For this, we need to know at which energy to look. Therefore, we estimate the mean energy of our spectrum using our measurements. For this we will:
1. Choose regions of interest containing only the basis materials we are interested in (here HA and water)
2. Calculate the mean attenuation coefficient in this ROI
3. Find where the mean attenuation coefficient of the image intersects with the theoretical attenuation coefficient --> mean energy of the spectrum

Here we need the mass absorption coefficients at the low and the high energy value, as well as the mass density you can use the xcom database from the NIST to look this up. https://www.nist.gov/pml/x-ray-mass-attenuation-coefficients or https://physics.nist.gov/cgi-bin/Xcom/xcom2 .

In [17]:
# Choose a region of interest (ROI) that only contains resin (water)
# then calculate the mean attenuation coefficient in this ROI
water_le = np.mean(LE_mu[230:320,460:580])
water_he = np.mean(HE_mu[230:320,460:580])
# now choose a region that contains HA (e.g. the highest absorbing one)
ha1200_le = np.mean(LE_mu[448:494,636:684])
ha1200_he = np.mean(HE_mu[448:494,636:684])

energy_bins = np.linspace(20,140,121) # a range of possible energies
att_array = np.zeros((len(energy_bins),2))

# get the theoretical linear attenuation coefficients 
for i, energy in enumerate(energy_bins):
    att_array[i,0] = xraylib.CS_Total_CP('Ca5P3O13H', int(energy))* 1.2 # multiplication with density since the function itself returns the mass absorption coeff., in the highest absorbing rod we have 1200mg/cm³ density
    att_array[i,1] = xraylib.CS_Total_CP('H2O', int(energy))*1.00
    
    
# now plot one of the images and show your ROI
# also plot the theoretical attenuation coefficient of hydroxyapatite together with the mean values you calculated --> where they intersect, you get your mean energy 
from matplotlib.patches import Rectangle
ROI_w = Rectangle((460, 320), 580-460, -(320-230), fill = False, linewidth = 2., edgecolor = 'r') # rectangle patch for water ROI
ROI_ha = Rectangle((636, 494), 684-636, -(494-448), fill = False, linewidth = 2., edgecolor = 'r') # rectangle patch for HA ROI
fig, ax = plt.subplots(1,2)
plt.suptitle('Energy Dependent Attenuation Coefficients')
im = ax[0].imshow(LE_mu, cmap = 'gray', vmin = 0.2)
ax[0].title.set_text('ROI in LE image')
ax[0].add_patch(ROI_ha)
'''
ax[1] = plt.plot(energy_bins, att_array[:,1], label='Water')
ax[1] = plt.plot([energy_bins[0],energy_bins[-1]], [water_le, water_le], label='Water ROI LE')
ax[1] = plt.plot([energy_bins[0],energy_bins[-1]], [water_he, water_he], label='Water ROI HE')
'''
# HA 1200 mg/ml
ax[1] = plt.plot(energy_bins, att_array[:,0], label='HA1200') # we add water because it is not pure HA that we measure
ax[1] = plt.plot([energy_bins[0],energy_bins[-1]], [ha1200_le, ha1200_le], label='Rod LE')
ax[1] = plt.plot([energy_bins[0],energy_bins[-1]], [ha1200_he, ha1200_he], label='Rod HE')

plt.yscale('log')
plt.xlabel('Energy [keV]')
plt.ylabel('Attenuation coefficient [1/cm]')
plt.legend()
plt.tight_layout()
plt.show()

# find the intersection of the mean values with the theoretical curve and estimate the energy
# mean low energy should be:
mean_le_estimate = 26.7 # keV # 27.7 for HA1200 Rod
mean_he_estimate = 57.5

In a perfect world, the measured linear attenuation coefficients should intersect with the theoretical curve. If, for example, the HE value does not intersect, we can use our source spectrum to calculate a theoretical mean energy for the linear attenuation coefficient.
The high energy measurement was taken at 140keV with a tin filter (0.02cm thickness).

In [88]:
#calculate the mean energy
# mean high energy should be:
#mean_he_estimate = 73.8 # keV # 75.5 for HA1200 Rod

In [26]:
# next we build our coefficient matrix
# get the theoretical attenuation values necessary from NIST / xraylib
# these are now mass attenuation coefficients!
mu_1_LE = xraylib.CS_Total_CP('Ca5P3O13H', mean_le_estimate) * 1.2 # HA with density 1.2g/cm^3
mu_1_HE = xraylib.CS_Total_CP('Ca5P3O13H', mean_he_estimate) * 1.2


mu_2_LE = xraylib.CS_Total_CP('H2O', mean_le_estimate) # Water
mu_2_HE = xraylib.CS_Total_CP('H2O', mean_he_estimate)

# As we are performing a two material decomposition, the coeff matrix is 2x2
coeff = np.array([[mu_1_LE, mu_2_LE],[mu_1_HE, mu_2_HE]])
print(coeff.shape)

(2, 2)


Concerning choosing the density for the attenuation coefficients in the coefficient matrix:
The volume fractions you reconstruct in the decomposition must be viewed as "this pixel behaves as if it contained [volume fraction] of material X that has the density $\rho$.".

### Step 3: Perform decomposition
Perform the decomposition the same way you performed it for the simulated phantom. Take a look at the resulting images. Are they plausible?

In [27]:
measurements = np.zeros((LE.shape[0], LE.shape[1],2)) # build an empty array in the right shape
measurements[:,:, 0] = LE_mu # assign the images to the last axis
measurements[:,:, 1] = HE_mu
print(measurements.shape, LE_mu.shape)

# This is the step where all the numpy magic happens. 
# If you want to understand the notation, please look at the numpy documentation and make yourself familiar with its array broadcasting rules.
decomp=np.linalg.solve(coeff[None, None], measurements)

# its easier to look at the images individually.
decomp_mat_1 = decomp[:,:,0]
decomp_mat_2 = decomp[:,:,1]

# checking out the decomposition images
#%matplotlib notebook 
fig, axs = plt.subplots(1, 2)
pcm1 = axs[0].imshow(decomp_mat_1, cmap='gray', vmin=0, vmax=1.2)
fig.colorbar(pcm1, ax=axs[0], fraction=0.046, pad=0.04)
axs[0].axis('off')
axs[0].title.set_text('Material Image 1')
pcm2 = axs[1].imshow(decomp_mat_2, cmap='gray', vmin=0, vmax=1.5)
fig.colorbar(pcm2, ax=axs[1], fraction=0.046, pad=0.04)
axs[1].axis('off')
axs[1].title.set_text('Material Image 2')
fig.tight_layout()
plt.show()


(973, 973, 2) (973, 973)


You can either use the surrounding resin to estimate the mean energy or the HA rod. 

Concerning choosing the density for the attenuation coefficients in the coefficient matrix:
The volume fractions you reconstruct in the decomposition must be viewed as "this pixel behaves as if it contained [volume fraction] of material X that has the density $\rho$.".

